In [3]:
import importlib
from utils import SQLConnect, SimFin_DataRequestor
import requests

import pandas as pd
API_KEY = 'N8XsZtAZtjMH8aUVK0d4A6HXm152V0TF'


In [6]:
sqlconnect = SQLConnect.SQLConnect()

In [7]:
balance_sheet_querystr = """
SELECT 
	simId
	, ticker
	, fiscal_year
	, period_type
	, [Invested Capital] = SUM(CASE WHEN t.StandardisedName IN ('Total Assets', 'Short Term Debt', 'Interest & Dividends Payable') THEN t.valueChosen ELSE 0 END) -
							SUM(CASE WHEN t.StandardisedName IN ('Total Current Liabilities') THEN t.valueChosen ELSE 0 END)
FROM SimFin.balancesheet t (nolock)
WHERE
	period_type = 'fy'
GROUP BY 
	simId
	, ticker
	, fiscal_year
	, period_type
"""

In [8]:
df_balancesheet = pd.read_sql(balance_sheet_querystr, sqlconnect.conn)

In [6]:
income_statement_querystr = """
SELECT 
	simId
	, ticker
	, fiscal_year
	, period_type
	, [EBIT] = SUM(CASE WHEN t.StandardisedName = 'Operating Income (Loss)' THEN t.valueChosen ELSE 0 END) 
FROM SimFin.incomestatement t (nolock)
WHERE
	period_type = 'fy'
GROUP BY 
	simId
	, ticker
	, fiscal_year
	, period_type
"""

In [9]:
df_incomestatement = pd.read_sql(income_statement_querystr, sqlconnect.conn)

In [18]:
def generate_df_roic(tickers):
    """
    Create a dataframe with ROIC info from the raw balance sheet and income statement data
    in the SQL database
    """
    df_result = pd.DataFrame()
        
    for ticker in tickers:
        df_ticker_bs = df_balancesheet[df_balancesheet['ticker'] == ticker].sort_values(by='fiscal_year')
        df_ticker_bs['Avg_Invested_Capital'] = (df_ticker_bs['Invested Capital'] + df_ticker_bs['Invested Capital'].shift()) / 2

        df_ticker_is = df_incomestatement[df_incomestatement['ticker'] == ticker].sort_values(by='fiscal_year')

        df_ticker_roic = df_ticker_bs.merge(df_ticker_is)
        df_ticker_roic['ROIC'] = df_ticker_roic['EBIT'] / df_ticker_roic['Avg_Invested_Capital']
        
        if df_result.empty:
            df_result = df_ticker_roic.dropna()
        else:
            df_result = df_result.append(df_ticker_roic.dropna())
        
    return df_result

In [26]:
generate_df_roic(tickers = ['AAPL'])

,simId,ticker,fiscal_year,period_type,Invested Capital,Avg_Invested_Capital,EBIT,ROIC
1,111052,AAPL,2009,fy,35995000000,3.040250e+10,11740000000,0.386152
2,111052,AAPL,2010,fy,54461000000,4.522800e+10,18385000000,0.406496
3,111052,AAPL,2011,fy,88401000000,7.143100e+10,33790000000,0.473044
4,111052,AAPL,2012,fy,137522000000,1.129615e+11,55241000000,0.489025
5,111052,AAPL,2013,fy,163342000000,1.504320e+11,48999000000,0.325722
6,111052,AAPL,2014,fy,174699000000,1.690205e+11,52503000000,0.310631
7,111052,AAPL,2015,fy,220734000000,1.977165e+11,71230000000,0.360263
8,111052,AAPL,2016,fy,254285000000,2.375095e+11,60024000000,0.252723
9,111052,AAPL,2017,fy,292978000000,2.736315e+11,61344000000,0.224185


In [27]:
simId = 111052  # atvi
prices_querystr = """
SELECT *
FROM SimFin.vw_fyearEnd_Prices
WHERE simId = %s""" % simId

In [28]:
pd.read_sql(prices_querystr, sqlconnect.conn)

,simId,ticker,fyearEnd,fyearEnd_Month,fyear,filing_date,closeAdj_at_filing_date
0,111052,AAPL,9,1,2007,2007-11-30,23.07
1,111052,AAPL,9,1,2008,2008-11-28,11.73
2,111052,AAPL,9,1,2009,2009-11-30,25.31
3,111052,AAPL,9,1,2010,2010-11-30,39.39
4,111052,AAPL,9,1,2011,2011-11-30,48.39
5,111052,AAPL,9,1,2012,2012-11-30,74.77
6,111052,AAPL,9,1,2013,2013-11-29,72.81
7,111052,AAPL,9,1,2014,2014-11-28,111.30
8,111052,AAPL,9,1,2015,2015-11-30,112.59
9,111052,AAPL,9,1,2016,2016-11-30,107.53


In [8]:
tickers = [ticker for ticker in df_balancesheet.ticker.unique()][:200]
x = generate_df_roic(tickers)
x.shape

NameError: name 'df_incomestatement' is not defined

In [12]:
year = 2014

roic = x[x['fiscal_year'] == year].sort_values(by='ROIC', ascending = False)
print(roic.shape)
roic

(198, 8)


,simId,ticker,fiscal_year,period_type,Invested Capital,Avg_Invested_Capital,EBIT,ROIC
4,167827,PRGB,2014,fy,-2279635,-1.328174e+06,-10693148,8.051017
4,663685,FONU,2014,fy,-243348,-6.260805e+05,-1067804,1.705538
4,663520,PMXO,2014,fy,-370907,-3.382880e+05,-300379,0.887939
3,69772,SLNM,2014,fy,-1968000,-5.491500e+06,-3901000,0.710371
5,104458,PII,2014,fy,1226653000,1.083676e+09,714690000,0.659505
6,256569,TJX,2014,fy,7059116000,6.871148e+09,3606501000,0.524876
4,346874,NTRI,2014,fy,58225000,5.658050e+07,29244000,0.516857
4,423305,PPC,2014,fy,2372423000,2.435830e+09,1203115000,0.493924
6,86689,SBUX,2014,fy,7714200000,6.926800e+09,2792600000,0.403159
6,339017,FAST,2014,fy,2073749000,1.954850e+09,786626000,0.402397


In [13]:
generate_df_roic(['TJX'])

,simId,ticker,fiscal_year,period_type,Invested Capital,Avg_Invested_Capital,EBIT,ROIC
1,256569,TJX,2009,fy,4571346000,4.188362e+09,1991071000,0.475382
2,256569,TJX,2010,fy,4841369000,4.706358e+09,2191679000,0.465685
3,256569,TJX,2011,fy,5221152000,5.031260e+09,2447062000,0.486372
4,256569,TJX,2012,fy,5751259000,5.486206e+09,3106526000,0.566243
5,256569,TJX,2013,fy,6683179000,6.217219e+09,3350570000,0.538918
6,256569,TJX,2014,fy,7059116000,6.871148e+09,3606501000,0.524876
7,256569,TJX,2015,fy,7088201000,7.073658e+09,3704700000,0.523732
8,256569,TJX,2016,fy,8126152000,7.607176e+09,3849523000,0.506038
9,256569,TJX,2017,fy,8932478000,8.529315e+09,3987426000,0.467497


In [22]:
from urllib.request import Request, urlopen
import urllib
import requests

In [15]:
simId = 256569

In [15]:
url  = 'https://simfin.com/api/v1/companies/id/' + repr(simId) + '/shares/prices?api-key=' + API_KEY
task =  {'start': '2017-01-01', "end": '2017-12-31'}

NameError: name 'simId' is not defined

In [96]:
response = requests.get(url, task)
result = response.json()

In [18]:
result

{'shareClassId': 617993,
 'shareClassName': 'Common shares',
 'shareClassType': 'common',
 'priceData': [{'date': '2017-12-29',
   'closeAdj': '75.50',
   'splitCoef': '1.00'},
  {'date': '2017-12-28', 'closeAdj': '75.43', 'splitCoef': '1.00'},
  {'date': '2017-12-27', 'closeAdj': '76.01', 'splitCoef': '1.00'},
  {'date': '2017-12-26', 'closeAdj': '76.48', 'splitCoef': '1.00'},
  {'date': '2017-12-22', 'closeAdj': '75.30', 'splitCoef': '1.00'},
  {'date': '2017-12-21', 'closeAdj': '75.35', 'splitCoef': '1.00'},
  {'date': '2017-12-20', 'closeAdj': '74.85', 'splitCoef': '1.00'},
  {'date': '2017-12-19', 'closeAdj': '75.11', 'splitCoef': '1.00'},
  {'date': '2017-12-18', 'closeAdj': '74.53', 'splitCoef': '1.00'},
  {'date': '2017-12-15', 'closeAdj': '73.24', 'splitCoef': '1.00'},
  {'date': '2017-12-14', 'closeAdj': '72.50', 'splitCoef': '1.00'},
  {'date': '2017-12-13', 'closeAdj': '72.98', 'splitCoef': '1.00'},
  {'date': '2017-12-12', 'closeAdj': '72.70', 'splitCoef': '1.00'},
  {'dat